<h1>Getting engagement data for tweets Twitter Climate Change sentiment dataset</h1>

In [ ]:
import pandas as pd
import numpy as np
import tweepy 

<h2>Load Data</h2>

In [ ]:
tweets = pd.read_csv("../twitter_sentiment_data.csv")

In [ ]:
tweets.head()

<h2>Extract Engagement</h2>

In [ ]:
bearer_token = "AAAAAAAAAAAAAAAAAAAAAHc8OQEAAAAAcU%2FIXW1wjJ0h%2FpQxvRKbYDyhJVI%3DntLxP3rMhKb9yTzTqLekvbsl2bP4F1OCRN4DLw3JPSbn9XzbDq"
client = tweepy.Client(bearer_token)

In [ ]:
def extract_entities(t):
    keys = list(t.data.entities.keys())
    if 'mentions' in keys:
        mentioned_users = [i['username'] for i in t.data.entities['mentions']]
    else:
        mentioned_users = None
    if 'hashtags' in keys:
        hashtags = [i['tag'] for i in t.data.entities['hashtags']]
    else: 
        hashtags = None
    if 'annnotations' in keys:
        annotations = [i['annotation'] for i in t.data.entities['annotations']]
    else: 
        annotations = None
    return (mentioned_users, hashtags, annotations)
        

In [ ]:
def get_tweet_data(t):
    t = client.get_tweet(t, tweet_fields=['entities', 'public_metrics', 'author_id'])
    if t.data == None:
        return None
    #identifiers
    tweet_id = t.data.id
    tweeter_id = t.data.author_id
    #get tweet attributes
    if getattr(s.data, 'public_metrics') != None:
        metrics = t.data.public_metrics
        retweets = metrics['retweet_count']
        replies = metrics['reply_count']
        likes = metrics['like_count']
        quote_tweets = metrics['quote_count']
    else:
        retweets = None
        replies = None
        likes = None
        quote_tweets = None
    #get tweet entities
    if getattr(t.data, 'entities'):
        mentioned_users, hashtags, annotations = extract_entities(t)
    else:
        mentioned_users, hashtags, annotations = None, None, None
    data = np.array([tweet_id, tweeter_id, retweets, replies, likes, quote_tweets, mentioned_users,
                     hashtags, annotations], dtype=object)
    return data

In [ ]:
tweets = tweets['tweetid'].values
engagement_data = np.array(list(map(get_tweet_data, sample)), dtype=object)